In [61]:
import pandas as pd
ezh_pk='https://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76626/suppl/GSE76626_ChIP-Seq_RPM.txt.gz' ## hg38
data='/labs/mpsnyder/moqri/data/index/'


In [33]:
l=range(4,7)
dics={
      'y1':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.YD1.mincov0.txt',
      'y2':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.YD2.mincov0.txt',
      'y3':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.YD3.mincov0.txt',
      'y4':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.YD4.mincov0.txt',
      'y5':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.YD5.mincov0.txt',
      'y6':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.YD6.mincov0.txt',
      'o1':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.OD1.mincov0.txt',
      'o2':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.OD2.mincov0.txt',
      'o3':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.OD3.mincov0.txt',
      'o4':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.OD4.mincov0.txt',
      'o5':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.OD5.mincov0.txt',
      'o6':'https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/per_donor/methylcall.CpG.OD6.mincov0.txt',
     }
dfs={}
for key, value in dics.items():
    if int(key[1:]) in l:
        dfs[key]=pd.read_table(value,index_col=0)
        print(key,end=',')

y4,y5,y6,o4,o5,o6,

In [34]:
ch=1
for key, value in dics.items():
    if int(key[1:]) in l:
        df=dfs[key]
        df=df[df.chr=='chr'+str(ch)]
        print(df.shape)
        dfs[key]=df

(435164, 6)
(374696, 6)
(361382, 6)
(374694, 6)
(419160, 6)
(385737, 6)


In [35]:
inds=[]
for key, value in dics.items():
    if int(key[1:]) in l:
        df=dfs[key]
        inds.append(df.index)

In [36]:
inds=set().union(*inds)
len(inds)

784583

In [37]:
dg=pd.DataFrame()
dg.index=inds

In [38]:
for key, value in dics.items():
    if int(key[1:]) in l:
        df=dfs[key]
        df['m_'+key]=df.freqC*df.coverage/100
        df['u_'+key]=df.freqT*df.coverage/100
        df=df.drop(['chr','base','strand','coverage','freqC','freqT'],1)
        dfs[key]=df

<ipython-input-38-ee1f8acb7556>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop(['chr','base','strand','coverage','freqC','freqT'],1)


In [39]:
for key, value in dics.items():
    if int(key[1:]) in l:
        df=dfs[key]
        dg=dg.merge(dfs[key],left_index=True,right_index=True,how='left')
dg.shape

(784583, 12)

In [40]:
dg['uy']=0
dg['my']=0
dg['uo']=0
dg['mo']=0
for i in l:
    dg['uy']=dg['uy']+dg['u_y'+str(i)]
    dg['my']=dg['my']+dg['m_y'+str(i)]
    dg['uo']=dg['uo']+dg['u_o'+str(i)]
    dg['mo']=dg['mo']+dg['m_o'+str(i)]

In [41]:
dg['by']=dg.my/(dg.my+dg.uy)
dg['bo']=dg.mo/(dg.mo+dg.uo)

In [42]:
dg['base']=dg.index.str[5:].astype(int)

In [43]:
dg=dg.sort_values('base')

In [44]:
#dg[dg.base>11044800].head(10)

In [45]:
dg=dg[['base','by','bo']]
dg['chr']='chr'+str(ch)
dg['base2']=dg['base']

In [63]:
dg[['chr','base','base2','by','bo']].to_csv(data+'hg19_mono.txt',index=False,header=False,sep='\t')

In [ ]:
!CrossMap.py bed hg19ToHg38.over.chain.gz {data}hg19_mono.txt {data}hg38_mono.txt

2021-11-19 11:16:10 [INFO]  Read the chain file "hg19ToHg38.over.chain.gz" 


In [48]:
df38=pd.read_table('hg38_mono.txt',header=None,names=['chr','b1','b2','by','bo'])
df38.shape

(784450, 5)

In [49]:
ezh=pd.read_table(ezh_pk,index_col=0)
ezh=ezh[ezh.Chromosome!='MT']
ezh=ezh[['Chromosome','Start','End','WT_EZH2']][ezh['WT_EZH2']>2]
ezh.columns=['chrom', 'chromStart', 'chromEnd', 'value']
ezh['chrom']=ezh.chrom.astype(str)
print(ezh.shape)
ez=ezh[ezh.chrom==str(ch)]
ez['r']=ez.apply(lambda x:range(x['chromStart'],x['chromEnd']),1)
ez=ez['r'].tolist()
ez=set([item for sublist in ez for item in sublist])

(1219, 4)


In [50]:
df38[df38['b1'].isin(ez)].dropna().shape

(5195, 5)

In [51]:
df38[df38['b1'].isin(ez)].dropna().mean()

<ipython-input-51-283bafdc63e2>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df38[df38['b1'].isin(ez)].dropna().mean()


b1    6.611045e+07
b2    6.611045e+07
by    4.854303e-02
bo    5.205158e-02
dtype: float64

In [52]:
df38[['by','bo']].dropna().mean()

by    0.390737
bo    0.393579
dtype: float64

In [53]:
v='38'
h9_bw='http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H9/tracks_hg'+v+'/Human_H9.meth.bw'
import pyBigWig

def f(cell,ch):
    df=cell.intervals("chr"+str(ch))
    print(len(df))
    df=pd.DataFrame(df)
    df.index=df[0]
    df=df[2]
    return(df)

In [54]:
h=pyBigWig.open(h9_bw)

In [55]:
h=f(h,ch)

2264861


In [56]:
hl=h[h<.5]
hl.shape

(343932,)

In [58]:
df38.dropna()[df38['b1'].isin(ez)].shape

<ipython-input-58-ffa10444b025>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df38.dropna()[df38['b1'].isin(ez)].shape


(5195, 5)

In [59]:
df38.dropna()[~(df38['b1']-1).isin(hl.index)][df38['b1'].isin(ez)].mean()

<ipython-input-59-c8a25cf27496>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df38.dropna()[~(df38['b1']-1).isin(hl.index)][df38['b1'].isin(ez)].mean()
<ipython-input-59-c8a25cf27496>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df38.dropna()[~(df38['b1']-1).isin(hl.index)][df38['b1'].isin(ez)].mean()


b1    6.631018e+07
b2    6.631018e+07
by    5.670614e-02
bo    6.000843e-02
dtype: float64